In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Data processing notebook
Here are the files for stock and news. Some basic processing were done to make it more concise

In [3]:
df_stock = pd.read_csv("SPX.csv")
df_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [7]:
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_stock.set_index('Date', inplace=True)
df_stock = df_stock.iloc[::-1]
df_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-04,3406.459961,3486.250000,3405.169922,3443.439941,3443.439941,4783040000
2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000
2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000


In [17]:
df_news = pd.read_csv("analyst_ratings_processed.csv")
df_news.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [ ]:
df_news = df_news.dropna(subset=['title'])  # doesn't seem necessary
del df_news['Unnamed: 0']  # this was the index
df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce', utc=True)
df_news = df_news.dropna(subset=['date'])
df_news.set_index('date', inplace=True)
df_news.head()

,title,stock
date,,
2020-06-05 14:30:00+00:00,Stocks That Hit 52-Week Highs On Friday,A
2020-06-03 14:45:00+00:00,Stocks That Hit 52-Week Highs On Wednesday,A
2020-05-26 08:30:00+00:00,71 Biggest Movers From Friday,A
2020-05-22 16:45:00+00:00,46 Stocks Moving In Friday's Mid-Day Session,A
2020-05-22 15:38:00+00:00,B of A Securities Maintains Neutral on Agilent...,A


In [19]:
df_news.shape, df_stock.shape

((1397891, 2), (23323, 6))

In [21]:
df_news.iloc[-1]

title    China Zenix Auto International Opens For Tradi...
stock                                                   ZX
Name: 2011-05-12 13:36:00+00:00, dtype: object

In [23]:
df_stock = df_stock[df_stock.index >= '2010-01-01']
df_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-04,3406.459961,3486.250000,3405.169922,3443.439941,3443.439941,4783040000
2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000
2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000


In [27]:
print(len(df_news['stock'].unique()))


6192
